In [ ]:
%pip install llama-index-llms-huggingface

In [ ]:
%pip install llama-index

In [3]:
from llama_index.core import VectorStoreIndex,SimpleDirectoryReader
from llama_index.llms.huggingface import HuggingFaceLLM

In [4]:
!mkdir data

In [5]:
documents=SimpleDirectoryReader("./data/").load_data()

In [6]:
print(documents) #reading the data

[Document(id_='70f2d329-f4de-484e-8434-2521eee1c424', embedding=None, metadata={'page_label': '1', 'file_name': 'attention all you need.pdf', 'file_path': '/content/data/attention all you need.pdf', 'file_type': 'application/pdf', 'file_size': 2215244, 'creation_date': '2024-04-11', 'last_modified_date': '2024-04-11'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, text='Provided proper attribution is provided, Google hereby grants permission to\nreproduce the tables and figures in this paper solely for use in journalistic or\nscholarly works.\nAttention Is All You Need\nAshish Vaswani∗\nGoogle Brain\navaswani@google.comNoam Shazeer∗\nGoogle Brain\nnoam@google.comNiki Parmar∗\nGoogle Research\nnikip@google.comJakob Uszkoreit∗\nGoogle Research\nusz@google.com\

In [7]:
# setup prompts - specific to StableLM
from llama_index.core import PromptTemplate

In [8]:

system_prompt = """<|SYSTEM|># You are a Q&A assistant. Your goal is to answer questions as
accurately as possible based on the instructions and context provided.
"""

# This will wrap the default prompts that are internal to llama-index
query_wrapper_prompt = PromptTemplate("<|USER|>{query_str}<|ASSISTANT|>")

In [ ]:

import torch

llm = HuggingFaceLLM(
    context_window=4096,
    max_new_tokens=256,
    generate_kwargs={"temperature": 0.7, "do_sample": False},
    system_prompt=system_prompt,
    query_wrapper_prompt=query_wrapper_prompt,
    tokenizer_name="mistralai/Mistral-7B-Instruct-v0.1",
    model_name="mistralai/Mistral-7B-Instruct-v0.1",
    device_map="auto",
    stopping_ids=[50278, 50279, 50277, 1, 0],
    tokenizer_kwargs={"max_length": 4096},
    # uncomment this if using CUDA to reduce memory usage
    model_kwargs={"torch_dtype": torch.float16}
)

In [ ]:
%pip install llama-index-embeddings-huggingface
%pip install llama-index-embeddings-instructor

In [11]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
embed_model=HuggingFaceEmbedding(model_name="sentence-transformers/all-mpnet-base-v2")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [12]:
from llama_index.core import VectorStoreIndex, ServiceContext

service_context = ServiceContext.from_defaults(
    chunk_size=1024,
    llm=llm,
    embed_model=embed_model
)

<ipython-input-12-5b4ac3d90986>:3: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context = ServiceContext.from_defaults(


In [13]:
index = VectorStoreIndex.from_documents(documents, service_context=service_context)

In [14]:
query_engine=index.as_query_engine()

In [15]:
query_engine.query("what is attention")

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:410: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Response(response='Attention is a mechanism used in natural language processing and machine learning to focus on relevant information in a given input. It is a way of weighting the importance of different parts of the input based on their relevance to the task at hand. Attention can be used in various ways, such as in sequence-to-sequence models, language translation, and text summarization. In the context of this document, attention is used in the Multi-Head Attention mechanism, which is a type of attention that involves multiple attention heads working in parallel to compute the attention weights for each value in the input.', source_nodes=[NodeWithScore(node=TextNode(id_='e599b455-e0cd-48dd-81d1-81ece12af692', embedding=None, metadata={'page_label': '14', 'file_name': 'attention all you need.pdf', 'file_path': '/content/data/attention all you need.pdf', 'file_type': 'application/pdf', 'file_size': 2215244, 'creation_date': '2024-04-11', 'last_modified_date': '2024-04-11'}, excluded_

In [17]:
sample=query_engine.query("what is Positional Encoding")

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:410: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [18]:
print(sample)

Positional Encoding is a technique used in the Transformer model architecture to inject information about the relative or absolute position of tokens in a sequence. It is added to the input embeddings at the bottoms of the encoder and decoder stacks. The positional encodings have the same dimension as the embeddings, so that the two can be summed. There are many choices of positional encodings, learned and fixed. In this work, the sinusoidal version was used because it may allow the model to extrapolate to sequence lengths longer than the ones encountered during training.
